# Notebook to Visualize and Compare Reference Data 

In [ ]:
# !pip install pandas_bokeh

In [43]:
import pandas as pd
import numpy as np
import boto3
import io

import pandas_bokeh
from bokeh.io import output_file, show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, ColumnDataSource, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.transform import transform
from bokeh.layouts import grid, row


import datetime as dt
from pytz import timezone

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import timedelta



In [44]:
pandas_bokeh.output_notebook()
# output_file(filename="custom_filename.html", title="Static HTML file")

Loading BokehJS ...

In [45]:
# rainpreddf=rainpreddf.reset_index()
# rainpreddf.set_index(pd.DatetimeIndex(rainpreddf.time), inplace=True)

## Process Apogee

In [46]:
def setdftime(df):
    '''set dataframe time to be time index'''
    df=df.reset_index()
    df['TIMESTAMP'] = pd.to_datetime(df.TIMESTAMP,  errors='coerce', format='%Y-%m-%d %H:%M:%S')
    df.set_index(pd.DatetimeIndex(df['TIMESTAMP']) , inplace=True)
    return df


In [47]:
APGdf=pd.read_csv('s3://arable-adse-dev/rain_classification_april_2022/SFRT_TW_ref/Apogee.csv', error_bad_lines=False, skiprows=3)
APGdf.columns=['TIMESTAMP','RECORD','BattV_Avg','PTemp_C_Avg','SWin_Avg','SWout_Avg','LWin_Avg','LWout_Avg','SWnet_Avg','LWnet_Avg','SWalbedo_Avg','NR_Avg']

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 17344: expected 12 fields, saw 23\n'
b'Skipping line 232262: expected 12 fields, saw 14\n'
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (4,5,6,7,8,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [48]:
APGdf=setdftime(APGdf)
# APGdf.set_index('TIMESTAMP', inplace=True)

In [49]:
#assign datatype to numeric
APGdf=APGdf.loc[:, APGdf.columns!='TIMESTAMP'].astype(float)
# APGdf=APGdf[~'TIMESTAMP'].astype(float)

In [50]:
features=['BattV_Avg','PTemp_C_Avg','SWin_Avg','SWout_Avg','LWin_Avg','LWout_Avg','SWnet_Avg','LWnet_Avg','SWalbedo_Avg','NR_Avg']

In [51]:
# for feature in features:
#     APGdf.loc['2022-08-1': '2022-08-12'][feature].plot(figsize=(12,3))
#     plt.show()

## Process Apogee_WNC

In [52]:
APGdfWNC=pd.read_csv('s3://arable-adse-dev/rain_classification_april_2022/SFRT_TW_ref/WNC_Apogee.csv', error_bad_lines=False, skiprows=3)
APGdfWNC.columns=['TIMESTAMP','RECORD','BattV_Avg','PTemp_C_Avg','SWin_Avg','SWout_Avg','LWin_Avg','LWout_Avg','SWnet_Avg','LWnet_Avg','SWalbedo_Avg','NR_Avg']

b'Skipping line 7161: expected 12 fields, saw 22\nSkipping line 8629: expected 12 fields, saw 22\nSkipping line 9347: expected 12 fields, saw 23\nSkipping line 14535: expected 12 fields, saw 17\nSkipping line 14863: expected 12 fields, saw 17\nSkipping line 17501: expected 12 fields, saw 17\n'


In [53]:
APGdfWNC.head()

,TIMESTAMP,RECORD,BattV_Avg,PTemp_C_Avg,SWin_Avg,SWout_Avg,LWin_Avg,LWout_Avg,SWnet_Avg,LWnet_Avg,SWalbedo_Avg,NR_Avg
0,1972-02-29 04:16:00,0,12.14,22.62,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN
1,1972-02-29 04:17:00,1,12.13,22.62,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN
2,1972-02-29 04:18:00,2,12.15,22.62,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN
3,1972-02-29 04:19:00,3,12.15,22.62,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN
4,1972-02-29 04:20:00,4,12.14,22.62,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN


In [54]:
APGdfWNC=setdftime(APGdfWNC)
# APGdfWNCWNC.set_index('TIMESTAMP', inplace=True)

In [55]:
#assign datatype to numeric
APGdfWNC=APGdfWNC.loc[:, APGdfWNC.columns!='TIMESTAMP'].astype(float)
# APGdfWNC=APGdfWNC[~'TIMESTAMP'].astype(float)

In [56]:
features=['BattV_Avg','PTemp_C_Avg','SWin_Avg','SWout_Avg','LWin_Avg','LWout_Avg','SWnet_Avg','LWnet_Avg','SWalbedo_Avg','NR_Avg']

## 1. Plot Apogee_WNC vs. Apogee TimeSeries

In [57]:
ax=[0]*len(features)
for i,feature in enumerate(features):
    ax[i] = figure(width=900, height=200, x_axis_type="datetime") 
    ax[i].line(APGdf.loc['2022-08-1': '2022-08-12'].index.values, APGdf.loc['2022-08-1': '2022-08-12'][feature], line_color='red', \
           legend_label="APG")
    ax[i].line(APGdfWNC.loc['2022-08-1': '2022-08-12'].index.values, APGdfWNC.loc['2022-08-1': '2022-08-12'][feature], line_color='green', \
           legend_label="APGWNC")
    ax[i].title.text = f'{feature} '

#     show(p)
#     save(p)

In [ ]:
res=[ax[i] for i in range(8)]
grid_layout = grid(res, nrows=4)

show(grid_layout)

In [59]:
#drop null value in TIMESTAMP

APGdf = APGdf[APGdf.index.notnull()]
APGdfWNC = APGdfWNC[APGdfWNC.index.notnull()]
APGdfWNC=APGdfWNC.sort_index()
APGdf=APGdf.sort_index()

In [60]:
## xplots
MergeAPGdfnAPGdfWXT=pd.merge_asof(APGdf, APGdfWNC, left_on='TIMESTAMP', right_on='TIMESTAMP', by='TIMESTAMP')

In [61]:
print(MergeAPGdfnAPGdfWXT.columns.tolist())

['TIMESTAMP', 'index_x', 'RECORD_x', 'BattV_Avg_x', 'PTemp_C_Avg_x', 'SWin_Avg_x', 'SWout_Avg_x', 'LWin_Avg_x', 'LWout_Avg_x', 'SWnet_Avg_x', 'LWnet_Avg_x', 'SWalbedo_Avg_x', 'NR_Avg_x', 'index_y', 'RECORD_y', 'BattV_Avg_y', 'PTemp_C_Avg_y', 'SWin_Avg_y', 'SWout_Avg_y', 'LWin_Avg_y', 'LWout_Avg_y', 'SWnet_Avg_y', 'LWnet_Avg_y', 'SWalbedo_Avg_y', 'NR_Avg_y']


In [62]:
features3= ['BattV_Avg', 'PTemp_C_Avg', 'SWin_Avg', 'SWout_Avg', 'LWin_Avg', 'LWout_Avg', 
            'SWnet_Avg', 'LWnet_Avg', 'SWalbedo_Avg', 'NR_Avg']
# MergeAPGdfnAPGdfWXT

## 2. Plot Apogee_WNC vs. Apogee Cross Plot

In [ ]:
p=[0]*len(features3)
    
for i, feature in enumerate(features3):
    p[i] = figure(width=200, height=200) 
#     f, p[i] = plt.subplots(figsize=(6, 6))
    p[i].scatter(x=eval(f'MergeAPGdfnAPGdfWXT.{feature}_x'),y=eval(f'MergeAPGdfnAPGdfWXT.{feature}_y'))
    
#     p[i] = MergeAPGdfnAPGdfWXT.plot_bokeh(kind="scatter", x=eval(f'{feature}_x'),y=eval(f'{feature}_y'),show_figure=False)
    p[i].title.text = f'{feature} '

    show(p[i])
    

# Process WXT Data

In [64]:


# WXT=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/WXT536.csv', error_bad_lines=False)
# WXT
WNCWXT=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/WNC_WXT536.csv', error_bad_lines=False)
WNCWXT

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 46430: expected 13 fields, saw 18\nSkipping line 47733: expected 13 fields, saw 14\nSkipping line 54389: expected 13 fields, saw 23\n'
b'Skipping line 66399: expected 13 fields, saw 20\n'
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,,,,,TOA5,1001,CR1000X,39339,CR1000X.Std.06.00,CPU:WNC_2minSYNC_2022_07_09.CR1X,29635,WNC_WXT536
TIMESTAMP,RECORD,WindDir_wxt,WS_ms_wxt_Avg,WS_ms_wxt_Max,WS_ms_wxt_TMx,WS_ms_wxt_Min,WS_ms_wxt_TMn,AirTC_wxt_Avg,RH_wxt_Avg,BP_wxt_Avg,Rain_mm_wxt_Avg,Rain_mm_wxt_Tot
TS,RN,Degrees,m/s,m/s,m/s,m/s,m/s,Deg C,%,kPa,mm,mm
NaN,NaN,Smp,Avg,Max,TMx,Min,TMn,Avg,Avg,Avg,Avg,Tot
1972-02-29 04:16:00,0,273,0.1,0.1,1972-02-29 04:15:10,0.1,1972-02-29 04:15:10,22,51.43,99.1,0,0
1972-02-29 04:17:00,1,316,0.1,0.1,1972-02-29 04:16:10,0.1,1972-02-29 04:16:10,22,51.37,99.1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-12 21:26:00,11144,176,2.733,4.3,2022-08-12 21:25:20,1.7,2022-08-12 21:25:10,21.37,59.95,101.4,0,0
2022-08-12 21:27:00,11145,196,1.917,3.2,2022-08-12 21:26:10,0.6,2022-08-12 21:26:30,21.3,59.68,101.4,0,0
2022-08-12 21:28:00,11146,215,1.217,2.9,2022-08-12 21:27:20,0.4,2022-08-12 21:27:40,21.28,59.95,101.4,0,0
2022-08-12 21:29:00,11147,222,1.583,3.0,2022-08-12 21:29:00,0.5,2022-08-12 21:28:50,21.32,59.88,101.4,0,0


## Process WXT

In [65]:
WXT=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/WXT536.csv', error_bad_lines=False, skiprows=4)
WXT.columns=['TIMESTAMP','RECORD','WindDir_wxt','WS_ms_wxt_Avg','WS_ms_wxt_Max','WS_ms_wxt_TMx','WS_ms_wxt_Min','WS_ms_wxt_TMn','AirTC_wxt_Avg','RH_wxt_Avg','BP_wxt_Avg','Rain_mm_wxt_Avg','Rain_mm_wxt_Tot']

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 232866: expected 13 fields, saw 21\n'
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,3,4,6,8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [66]:
WXT=setdftime(WXT)
# APGdfWNCWNC.set_index('TIMESTAMP', inplace=True)

In [67]:
#assign datatype to numeric
WXT=WXT[WXT.columns[~WXT.columns.isin(['TIMESTAMP', 'WS_ms_wxt_TMx', 'WS_ms_wxt_TMn'])]].astype(float)

In [68]:
features=['WindDir_wxt','WS_ms_wxt_Avg','WS_ms_wxt_Max','WS_ms_wxt_Min','AirTC_wxt_Avg','RH_wxt_Avg','BP_wxt_Avg','Rain_mm_wxt_Avg','Rain_mm_wxt_Tot']

## Process WXTWNC

In [69]:
WXTWNC=pd.read_csv('s3://arable-adse-dev/reference_SFRT_TW/WNC_WXT536.csv', error_bad_lines=False, skiprows=4)
WXTWNC.columns=['TIMESTAMP','RECORD','WindDir_wxt','WS_ms_wxt_Avg','WS_ms_wxt_Max','WS_ms_wxt_TMx','WS_ms_wxt_Min','WS_ms_wxt_TMn','AirTC_wxt_Avg','RH_wxt_Avg','BP_wxt_Avg','Rain_mm_wxt_Avg','Rain_mm_wxt_Tot']

b'Skipping line 46430: expected 13 fields, saw 18\nSkipping line 47733: expected 13 fields, saw 14\nSkipping line 54389: expected 13 fields, saw 23\n'
b'Skipping line 66399: expected 13 fields, saw 20\n'


In [70]:
# WXTWNC.head()

In [71]:
WXTWNC=setdftime(WXTWNC)
# APGdfWNCWNC.set_index('TIMESTAMP', inplace=True)

In [72]:
#assign datatype to numeric
WXTWNC=WXTWNC[WXTWNC.columns[~WXTWNC.columns.isin(['TIMESTAMP', 'WS_ms_wxt_TMx', 'WS_ms_wxt_TMn'])]].astype(float)

In [78]:
features2=['WindDir_wxt', 'WS_ms_wxt_Avg', 'WS_ms_wxt_Max', 
           'WS_ms_wxt_Min', 'AirTC_wxt_Avg', 'RH_wxt_Avg', 'BP_wxt_Avg', 'Rain_mm_wxt_Avg', 'Rain_mm_wxt_Tot']

## 3. Plot WXT vs. WXTNC TimeSeries

In [ ]:
p=[0]*len(features2)
for i, feature in enumerate(features2):
    p[i] = figure(width=900, height=200) 
    p[i].line(WXT.loc['2022-08-1': '2022-08-2'].index.values, WXT.loc['2022-08-1': '2022-08-2'][feature], line_color='red', \
           legend_label="WXT")
    p[i].line(WXTWNC.loc['2022-08-1': '2022-08-2'].index.values, WXTWNC.loc['2022-08-1': '2022-08-2'][feature], line_color='green', \
           legend_label="WXTWNC")
    p[i].title.text = f'{feature} '

#     show(p)

In [87]:
res=[p[i] for i in range(9)]
grid_layout = grid(res, nrows=5)

show(grid_layout)

In [88]:
#drop null value in TIMESTAMP

WXT = WXT[WXT.index.notnull()]
WXTWNC = WXTWNC[WXTWNC.index.notnull()]
WXTWNC=WXTWNC.sort_index()
WXT=WXT.sort_index()

In [89]:
## xplots
MergeWXTnWXTWNC=pd.merge_asof(WXT, WXTWNC, left_on='TIMESTAMP', right_on='TIMESTAMP', by='TIMESTAMP')

In [90]:
feature2=[c for c in MergeWXTnWXTWNC.columns.tolist() if c not in ['TIMESTAMP', 'index_x', 'index_y' , 'RECORD_x', 'RECORD_y']]

In [91]:
print(MergeWXTnWXTWNC.columns.tolist())

['TIMESTAMP', 'index_x', 'RECORD_x', 'WindDir_wxt_x', 'WS_ms_wxt_Avg_x', 'WS_ms_wxt_Max_x', 'WS_ms_wxt_Min_x', 'AirTC_wxt_Avg_x', 'RH_wxt_Avg_x', 'BP_wxt_Avg_x', 'Rain_mm_wxt_Avg_x', 'Rain_mm_wxt_Tot_x', 'index_y', 'RECORD_y', 'WindDir_wxt_y', 'WS_ms_wxt_Avg_y', 'WS_ms_wxt_Max_y', 'WS_ms_wxt_Min_y', 'AirTC_wxt_Avg_y', 'RH_wxt_Avg_y', 'BP_wxt_Avg_y', 'Rain_mm_wxt_Avg_y', 'Rain_mm_wxt_Tot_y']


## 4. Plot WXT vs. WXTNC Crossplots

In [ ]:
feature2=['WindDir_wxt', 'WS_ms_wxt_Avg', 'WS_ms_wxt_Max', 'WS_ms_wxt_Min', 'AirTC_wxt_Avg', 'RH_wxt_Avg', 'BP_wxt_Avg', 'Rain_mm_wxt_Avg', 'Rain_mm_wxt_Tot']
ay=[0]*len(features2)
for i, feature in enumerate(features2):
    ay[i]= figure(width=400, height=400, x_axis_type="datetime")
    ay[i].scatter(x=eval(f'MergeWXTnWXTWNC.{feature}_x'), y=eval(f'MergeWXTnWXTWNC.{feature}_y'))
#     p.line(x=[eval(f'MergeWXTnWXTWNC.{feature}_x.min()'), eval(f'MergeWXTnWXTWNC.{feature}_y.min()')],
#            y=[eval(f'MergeWXTnWXTWNC.{feature}_x.may()'), eval(f'MergeWXTnWXTWNC.{feature}_y.may()')])
    ay[i].title.text = f'{feature}'

    show(ay[i])
# res=[ay[i] for i in range(len(feature2))]
# grid_layout = grid(res, nrows=4)

# show(grid_layout)